In [3]:
import numpy as np
import matplotlib.colors as mcolors
import plotly.graph_objects as go
import pandas as pd

VERY_BAD_COLOR = "#f04f0a"
BAD_COLOR = "#e89e1e"
OK_COLOR = "#abb53f"
GOOD_COLOR = "#208c6f"

THRESHOLD = 90
DANGER_THRESHOLD = 80


def minmax_scaler(x, from_range=None, to_range=None, trim=False):
    """Scales a numpy array to the range [a, b] using min-max normalization.
    Ref: https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)
    """
    _min = np.nanmin(x)
    _max = np.nanmax(x)
    from_range = from_range or (_min, _max)
    to_range = to_range or (_min, _max)

    if from_range[0] == from_range[1]:
        raise ValueError(f"Values in `from_range` must be different. Got {from_range}")

    _is_arr = True
    try:
        len(x)
    except TypeError:
        _is_arr = False

    if _is_arr:
        return [_minmax_scaler(i, from_range, to_range, trim) for i in x]
    else:
        return _minmax_scaler(x, from_range, to_range, trim)


def _minmax_scaler(x, from_range, to_range, trim):
    """Scale a number from a given range to a new range using min-max normalization.
    Ref: https://en.wikipedia.org/wiki/Feature_scaling#Rescaling_(min-max_normalization)
    """
    old_low, old_high = from_range
    new_low, new_high = to_range
    if trim:
        if x < old_low:
            x = old_low
        elif x > old_high:
            x = old_high
    return new_low + (((x - old_low) * (new_high - new_low)) / (old_high - old_low))


def rgb_arr(n, very_bad_coef=3, logspace_base=10):
    """
    Usage Note
    ----------
    Increase `very_bad_coef` and/or `logspace_base` to increase the
    proportion of "bad" colors.
    """
    cmap = mcolors.LinearSegmentedColormap.from_list(
        "custom_linear_cmap",
        [VERY_BAD_COLOR] * very_bad_coef + [BAD_COLOR] + [OK_COLOR] + [GOOD_COLOR],
        N=n,
    )
    values = [
        cmap(i)[:3]
        for i in minmax_scaler(
            np.logspace(0, 1, n, base=logspace_base), to_range=(0, 1)
        )
    ]
    return [f"rgb({int(r * 255)},{int(g * 255)},{int(b * 255)})" for r, g, b in values]

In [4]:
_n = 100
data = (
    [i / _n for i in range(_n)] + [1, np.nan, 1] + [i / _n for i in range(_n)][::-1]
)  # [0, 0.2, 0.5, np.nan, 0.8, 1, 0.8, 0.8, np.nan, 0.1, 0, 1]
dt = list(range(len(data)))

dff = pd.DataFrame({"Date": dt, "Overall Behavior Score": data})
scale_factor = 100  # scale to 100 to report percentages
sample_size = len(dff)
dff = dff.sort_values(by="Date").reset_index(drop=True)
dff["Behavior Score (%)"] = dff["Overall Behavior Score"] * scale_factor
rgb_indices = (
    dff["Overall Behavior Score"]
    .fillna(0)
    .apply(minmax_scaler, from_range=(0, 1), to_range=(0, sample_size - 1), trim=True)
).astype(int)
rgb = rgb_arr(
    n=sample_size, very_bad_coef=5, logspace_base=20
)  # rgb_arr(n=sample_size)
bar_color = [rgb[i] for i in rgb_indices]
dff["rgb_indices"] = rgb_indices
dff["bar_color"] = bar_color
student_mean = dff["Behavior Score (%)"].mean()
dff[["Behavior Score (%)", "rgb_indices", "bar_color"]].head(10)

,Behavior Score (%),rgb_indices,bar_color
0,0.0,0,"rgb(240,79,10)"
1,1.0,2,"rgb(240,79,10)"
2,2.0,4,"rgb(240,79,10)"
3,3.0,6,"rgb(240,79,10)"
4,4.0,8,"rgb(240,79,10)"
5,5.0,10,"rgb(240,79,10)"
6,6.0,12,"rgb(240,79,10)"
7,7.0,14,"rgb(240,79,10)"
8,8.0,16,"rgb(240,79,10)"
9,9.0,18,"rgb(240,79,10)"


In [5]:
fig = go.Figure()
x_data = list(range(sample_size))
fig.add_trace(
    go.Bar(
        x=dff["Date"],
        y=dff["Behavior Score (%)"],
        marker={"color": dff["bar_color"]},
        showlegend=False,
    )
)
fig.add_shape(
    type="line",
    xref="paper",
    yref="y",
    x0=0,
    y0=THRESHOLD,
    x1=1,
    y1=THRESHOLD,
    line=dict(
        color="black",
        width=2,
        dash="dot",
    ),
    name=f"Class Expectation ({THRESHOLD:.0f}%)",
)
fig.add_trace(
    go.Scatter(
        x=dff["Date"],
        y=[student_mean] * dff.shape[0],
        mode="lines",
        line=dict(
            color="black" if student_mean >= THRESHOLD else "#f04f0a",
            width=2,
        ),
        name=f"Average Score ({student_mean:.2f}%)",
    )
)
fig.update_layout(
    height=400,
    title={
        "text": "Class Behavior",
    },
    xaxis_title="Class Day",
    yaxis_title="Score (%)",
    xaxis={"showgrid": False, "showticklabels": False},
    yaxis_range=[-5, 105],
    yaxis={"showgrid": False, "showticklabels": True},
    bargap=0,
    bargroupgap=0,
)

fig.show()

In [ ]:
from datetime import datetime, timedelta

today = datetime.now()

date_list = sorted(
    list(set([(today - timedelta(days=i)).replace(day=1).date() for i in range(400)]))
)
formatted_date_list = [i.strftime("%b-%y") for i in date_list]
formatted_date_list

['Nov-24',
 'Dec-24',
 'Jan-25',
 'Feb-25',
 'Mar-25',
 'Apr-25',
 'May-25',
 'Jun-25',
 'Jul-25',
 'Aug-25',
 'Sep-25',
 'Oct-25',
 'Nov-25',
 'Dec-25']

In [19]:
a = [0, 1, 2, 3, 4, 5, 6, 7, 8]
a[-3:]

[6, 7, 8]

In [20]:
dict(zip(date_list, formatted_date_list))

{datetime.date(2024, 11, 1): 'Nov-24',
 datetime.date(2024, 12, 1): 'Dec-24',
 datetime.date(2025, 1, 1): 'Jan-25',
 datetime.date(2025, 2, 1): 'Feb-25',
 datetime.date(2025, 3, 1): 'Mar-25',
 datetime.date(2025, 4, 1): 'Apr-25',
 datetime.date(2025, 5, 1): 'May-25',
 datetime.date(2025, 6, 1): 'Jun-25',
 datetime.date(2025, 7, 1): 'Jul-25',
 datetime.date(2025, 8, 1): 'Aug-25',
 datetime.date(2025, 9, 1): 'Sep-25',
 datetime.date(2025, 10, 1): 'Oct-25',
 datetime.date(2025, 11, 1): 'Nov-25',
 datetime.date(2025, 12, 1): 'Dec-25'}